In [1]:
!pip install rdkit
!pip install duckdb
!pip install pandas networkx
!pip install torch
!pip install torch-geometric

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import duckdb
from torch.utils.data import Dataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 31.4 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 58.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.1 MB/s eta 0:00:0000:010:01
/kaggle/input/leash-BELKA/sample_submission.csv
/kaggle/input/leash-BELKA/train.parquet
/kaggle/input/leash-BELKA/test.parquet
/kaggle/input/leash-BELKA/train.csv
/kaggle/input/leash-BELKA/test.csv


In [2]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
test_path = '/kaggle/input/leash-BELKA/test.parquet'

con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

df = df.drop(['buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles'], axis=1)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
df.sample(n=20)
# print(df)
# rows, col= smiles_df.shape
# print(f"Number of rows: {rows}, Number of rows: {col}")

,id,molecule_smiles,protein_name,binds
55726,72811836,COC(=O)c1sc(-c2ccc(Cl)cc2)cc1Nc1nc(Nc2ccc(F)c(...,BRD4,1
33921,81827666,CCOCCCNc1nc(NCC2CCC(C(=O)N[Dy])CC2)nc(Nc2ccc(N...,sEH,1
12228,289020876,CS(=O)(=O)Nc1ccc(-c2csc(Nc3nc(NCC4CC5(CC5)CO4)...,BRD4,0
32929,177373345,Cc1cccc2sc(Nc3nc(Nc4cc(I)ccc4C(=O)N[Dy])nc(Nc4...,HSA,1
47126,33338641,COc1cc(Nc2nc(NCc3cnc4n3CCOC4)nc(Nc3cc(F)c(F)cc...,HSA,1
58247,287088935,Cc1cc2cc(CNc3nc(NCC45CC6CC(CC(C6)C4)C5)nc(N4c5...,sEH,1
13742,120246176,COC(=O)c1scnc1Nc1nc(NCCC(=O)Nc2ncccc2C)nc(N[C@...,sEH,0
14474,106147583,O=C1CN(CCCNc2nc(NCCCN3CCNC(=O)C3)nc(N[C@@H](Cc...,sEH,0
53063,218163789,COc1ccc(Nc2nc(Nc3ccnc(C(=O)N[Dy])c3)nc(Nc3cc(C...,BRD4,1
47998,86592738,COc1ccccc1-c1nnc(Nc2nc(Nc3ccc(=O)n(C)c3)nc(N[C...,BRD4,1


In [4]:
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
from torch_geometric.data import Data

def smiles_to_graph(smiles, protein_encoding):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    AllChem.Compute2DCoords(mol)
    num_atoms = mol.GetNumAtoms()

    # Node features
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())
    atom_features = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)

    # Edge indices
    edge_indices = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_indices.append((i, j))
        edge_indices.append((j, i))
    edge_indices = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

    # Protein encoding
    protein_features = torch.tensor(protein_encoding, dtype=torch.float)

    return Data(x=atom_features, edge_index=edge_indices, protein=protein_features)

In [5]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
protein_encoded = encoder.fit_transform(df[['protein_name']])
df['protein_encoded'] = list(protein_encoded)

In [6]:
graph_data_list = []
for index, row in df.iterrows():
    graph = smiles_to_graph(row['molecule_smiles'], row['protein_encoded'])
    if graph is not None:
        graph.y = torch.tensor([row['binds']], dtype=torch.float)
        graph_data_list.append(graph)

In [7]:
def normalize_features(features):
    mean = features.mean(dim=0, keepdim=True)
    std = features.std(dim=0, keepdim=True)
    return (features - mean) / (std + 1e-6)

for data in graph_data_list:
    data.x = normalize_features(data.x)

In [8]:
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader

class MoleculeDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

dataset = MoleculeDataset(graph_data_list)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import BatchNorm1d

class GCNWithProtein(torch.nn.Module):
    def __init__(self, protein_dim):
        super(GCNWithProtein, self).__init__()
        self.conv1 = GCNConv(1, 128)
        self.conv2 = GCNConv(128, 128)
        self.conv3 = GCNConv(128, 128)
        self.bn1 = BatchNorm1d(128)
        self.bn2 = BatchNorm1d(128)
        self.bn3 = BatchNorm1d(128)
        self.fc1 = torch.nn.Linear(128 + protein_dim, 256)
        self.fc2 = torch.nn.Linear(256, 1)
        self.dropout = torch.nn.Dropout(p=0.3)
        
        # Initialize the weights of the linear layers
        self.init_weights()

    def init_weights(self):
        torch.nn.init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.fc2.weight)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        protein = data.protein

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)
        x = global_mean_pool(x, batch)

        protein = protein.view(batch.max().item() + 1, -1)

        x = torch.cat([x, protein], dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Get the protein dimension from one-hot encoding
protein_dim = len(encoder.categories_[0])
model = GCNWithProtein(protein_dim)

In [11]:
import torch.optim as optim

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = torch.nn.BCEWithLogitsLoss()

def train():
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y.view(-1, 1))
        loss.backward()
        optimizer.step()

def evaluate():
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            output = model(data)
            loss = criterion(output, data.y.view(-1, 1))
            val_loss += loss.item()
    return val_loss / len(val_loader)

for epoch in range(200):
    train()
    val_loss = evaluate()
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss}')

Epoch 1, Validation Loss: 0.6773141724268595
Epoch 2, Validation Loss: 0.6732469096183776
Epoch 3, Validation Loss: 0.6706325120925903
Epoch 4, Validation Loss: 0.6716855664253235
Epoch 5, Validation Loss: 0.6737308254241944
Epoch 6, Validation Loss: 0.6699365401268005
Epoch 7, Validation Loss: 0.6690623424847921
Epoch 8, Validation Loss: 0.6664311855634053
Epoch 9, Validation Loss: 0.6635685540835062
Epoch 10, Validation Loss: 0.6635009922981262
Epoch 11, Validation Loss: 0.6614815940856934
Epoch 12, Validation Loss: 0.6596655955314636
Epoch 13, Validation Loss: 0.6603262114524842
Epoch 14, Validation Loss: 0.6584972918828328
Epoch 15, Validation Loss: 0.6629143527348836
Epoch 16, Validation Loss: 0.6560667153994242
Epoch 17, Validation Loss: 0.658923126856486
Epoch 18, Validation Loss: 0.660286503314972
Epoch 19, Validation Loss: 0.6555044668515523
Epoch 20, Validation Loss: 0.6622659699122111
Epoch 21, Validation Loss: 0.6531050465901693
Epoch 22, Validation Loss: 0.6551520410378774

In [12]:
# Predict on new data
new_smiles = ['C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy]', 
              'C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy]', 
              'C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2ncn3CC(C)O)n1)C(=O)N[Dy]', 
              'C#CCCC[C@H](Nc1nc(NCc2ccncc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy]', 
              'C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2cc(C(F)(F)F)ccn2)n1)C(=O)N[Dy]']
new_proteins = ['BRD4', 'sEH', 'BRD4', 'HSA', 'HSA']
new_protein_df = pd.DataFrame(new_proteins, columns=['protein_name'])
new_protein_encoded = encoder.transform(new_protein_df)
new_graph_data_list = []

for smiles, protein_encoding in zip(new_smiles, new_protein_encoded):
    try:
        graph = smiles_to_graph(smiles, protein_encoding)
        if graph is not None:
            new_graph_data_list.append(graph)
    except Exception as e:
        print(f"Error processing SMILES '{smiles}': {e}")

new_loader = DataLoader(new_graph_data_list, batch_size=1, shuffle=False)

predictions = []
with torch.no_grad():
    for data in new_loader:
        logits = model(data)
        print(f"Logits: {logits}")  # Debugging print
        prediction = torch.sigmoid(logits).item()
        predictions.append(prediction)

# Print the predictions
for smiles, protein, pred in zip(new_smiles, new_proteins, predictions):
    print(f'SMILES: {smiles}, Protein: {protein}, Predicted Binding Affinity: {pred}')

Logits: tensor([[739.4473]])
Logits: tensor([[746.6923]])
Logits: tensor([[727.1858]])
Logits: tensor([[750.7463]])
Logits: tensor([[758.8871]])
SMILES: C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy], Protein: BRD4, Predicted Binding Affinity: 1.0
SMILES: C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy], Protein: sEH, Predicted Binding Affinity: 1.0
SMILES: C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ncnc3c2ncn3CC(C)O)n1)C(=O)N[Dy], Protein: BRD4, Predicted Binding Affinity: 1.0
SMILES: C#CCCC[C@H](Nc1nc(NCc2ccncc2)nc(Nc2ccc(C=C)cc2)n1)C(=O)N[Dy], Protein: HSA, Predicted Binding Affinity: 1.0
SMILES: C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2cc(C(F)(F)F)ccn2)n1)C(=O)N[Dy], Protein: HSA, Predicted Binding Affinity: 1.0
